In [1]:
import os, sys

In [2]:
import numpy as np
import pandas as pd
import copy

import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error

# Data

In [1]:
x_all = pd.read_csv("../Data/training_input_lat8cht.csv")
y_all = pd.read_csv("../Data/training_output_Z18GQCh.csv")

NameError: name 'pd' is not defined

### BenchMark

In [4]:
distance_inv = (1. / x_all.filter(regex = 'distance*', axis = 1)).values
values = x_all.filter(regex = 'value_*', axis = 1)
values = (values.fillna(0)).values

In [5]:
numer = (distance_inv * values).sum(axis = 1)
denom = (distance_inv * (values != 0)).sum(axis = 1)
predicts = numer / denom
x_all["y_pred"] = predicts
x_all['y'] = y_all["value"]

## Fill NaN

In [6]:
for col in x_all.filter(regex = 'value_*', axis = 1).columns:
    x_all[col].fillna(value = x_all.y_pred, inplace = True)

## Split

In [44]:
stats_id = np.unique(x_all["station_id"].values)
stats_groups = np.random.permutation(stats_id).reshape((5, -1))

l_train, l_val = [], []

for i in range(5):
    
    l_val.append(stats_groups[i])
    l_train.append(np.setdiff1d(stats_id, stats_groups[i]))

In [ ]:
train_stats = [num for num in stats_id if num not in val_stats]

In [8]:
x_train = x_all[x_all.station_id.isin(train_stats)].reset_index(drop = True)
x_val = x_all[x_all.station_id.isin(val_stats)].reset_index(drop = True)

In [82]:
#x_group = x_all.groupby("station_id")

# LightGBM

### Cross Validation

In [9]:
X_train = x_train.iloc[:, 2:-3].values
Y_train = (x_train["y"]).values

X_val = x_val.iloc[:, 2:-3].values
Y_val = (x_val["y"]).values

In [49]:
params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'root_mean_squared_error',
        'max_depth': 5, 
        'num_leaves': 25,
        'learning_rate': 0.01,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5
    }

In [50]:
l_train_acc, l_val_acc = [], []

for i in range(5):
    
    x_train = x_all[x_all.station_id.isin(l_train[i])].reset_index(drop = True)
    x_val = x_all[x_all.station_id.isin(l_val[i])].reset_index(drop = True)
    
    X_train = x_train.iloc[:, 2:-1].values
    Y_train = (x_train["y"]).values

    X_val = x_val.iloc[:, 2:-1].values
    Y_val = (x_val["y"]).values
    
    lgb_train = lgb.Dataset(X_train, Y_train)
    lgb_eval = lgb.Dataset(X_val, Y_val, reference = lgb_train)
    
    gbm = lgb.train(params,
                lgb_train,
                valid_sets = [lgb_eval],
                num_boost_round = 10000,
                early_stopping_rounds = 100)
    
    train_preds = gbm.predict(X_train)
    l_train_acc.append(mean_squared_log_error(Y_train, train_preds))
    
    val_preds = gbm.predict(X_val)
    l_val_acc.append(mean_squared_log_error(Y_val, val_preds))

[1]	valid_0's rmse: 17.5682
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 17.4757
[3]	valid_0's rmse: 17.3849
[4]	valid_0's rmse: 17.2954
[5]	valid_0's rmse: 17.2066
[6]	valid_0's rmse: 17.1197
[7]	valid_0's rmse: 17.0327
[8]	valid_0's rmse: 16.9482
[9]	valid_0's rmse: 16.8642
[10]	valid_0's rmse: 16.7819
[11]	valid_0's rmse: 16.7011
[12]	valid_0's rmse: 16.6218
[13]	valid_0's rmse: 16.543
[14]	valid_0's rmse: 16.4661
[15]	valid_0's rmse: 16.3902
[16]	valid_0's rmse: 16.3153
[17]	valid_0's rmse: 16.2418
[18]	valid_0's rmse: 16.1683
[19]	valid_0's rmse: 16.0967
[20]	valid_0's rmse: 16.0273
[21]	valid_0's rmse: 15.9578
[22]	valid_0's rmse: 15.8896
[23]	valid_0's rmse: 15.8218
[24]	valid_0's rmse: 15.7555
[25]	valid_0's rmse: 15.6886
[26]	valid_0's rmse: 15.6239
[27]	valid_0's rmse: 15.56
[28]	valid_0's rmse: 15.498
[29]	valid_0's rmse: 15.4367
[30]	valid_0's rmse: 15.3766
[31]	valid_0's rmse: 15.317
[32]	valid_0's rmse: 15.2588
[33]	valid_0's rmse: 15

[283]	valid_0's rmse: 12.0257
[284]	valid_0's rmse: 12.025
[285]	valid_0's rmse: 12.0252
[286]	valid_0's rmse: 12.0231
[287]	valid_0's rmse: 12.021
[288]	valid_0's rmse: 12.0202
[289]	valid_0's rmse: 12.0202
[290]	valid_0's rmse: 12.0191
[291]	valid_0's rmse: 12.0191
[292]	valid_0's rmse: 12.0206
[293]	valid_0's rmse: 12.02
[294]	valid_0's rmse: 12.0182
[295]	valid_0's rmse: 12.018
[296]	valid_0's rmse: 12.0201
[297]	valid_0's rmse: 12.0214
[298]	valid_0's rmse: 12.0218
[299]	valid_0's rmse: 12.0222
[300]	valid_0's rmse: 12.0222
[301]	valid_0's rmse: 12.0233
[302]	valid_0's rmse: 12.024
[303]	valid_0's rmse: 12.0225
[304]	valid_0's rmse: 12.024
[305]	valid_0's rmse: 12.0251
[306]	valid_0's rmse: 12.0241
[307]	valid_0's rmse: 12.0239
[308]	valid_0's rmse: 12.0242
[309]	valid_0's rmse: 12.0253
[310]	valid_0's rmse: 12.0258
[311]	valid_0's rmse: 12.0268
[312]	valid_0's rmse: 12.0273
[313]	valid_0's rmse: 12.0282
[314]	valid_0's rmse: 12.0278
[315]	valid_0's rmse: 12.028
[316]	valid_0's rm

[165]	valid_0's rmse: 10.9516
[166]	valid_0's rmse: 10.946
[167]	valid_0's rmse: 10.9391
[168]	valid_0's rmse: 10.9335
[169]	valid_0's rmse: 10.9265
[170]	valid_0's rmse: 10.9189
[171]	valid_0's rmse: 10.9129
[172]	valid_0's rmse: 10.9078
[173]	valid_0's rmse: 10.9005
[174]	valid_0's rmse: 10.8956
[175]	valid_0's rmse: 10.8913
[176]	valid_0's rmse: 10.8865
[177]	valid_0's rmse: 10.8809
[178]	valid_0's rmse: 10.878
[179]	valid_0's rmse: 10.871
[180]	valid_0's rmse: 10.8665
[181]	valid_0's rmse: 10.8602
[182]	valid_0's rmse: 10.8537
[183]	valid_0's rmse: 10.8469
[184]	valid_0's rmse: 10.8431
[185]	valid_0's rmse: 10.8381
[186]	valid_0's rmse: 10.8328
[187]	valid_0's rmse: 10.8292
[188]	valid_0's rmse: 10.8244
[189]	valid_0's rmse: 10.8215
[190]	valid_0's rmse: 10.8158
[191]	valid_0's rmse: 10.8132
[192]	valid_0's rmse: 10.81
[193]	valid_0's rmse: 10.8034
[194]	valid_0's rmse: 10.7991
[195]	valid_0's rmse: 10.7945
[196]	valid_0's rmse: 10.7898
[197]	valid_0's rmse: 10.7852
[198]	valid_0's

[449]	valid_0's rmse: 10.5225
[450]	valid_0's rmse: 10.5245
[451]	valid_0's rmse: 10.5245
[452]	valid_0's rmse: 10.5245
[453]	valid_0's rmse: 10.525
[454]	valid_0's rmse: 10.5248
[455]	valid_0's rmse: 10.5248
[456]	valid_0's rmse: 10.5248
[457]	valid_0's rmse: 10.5249
[458]	valid_0's rmse: 10.5257
[459]	valid_0's rmse: 10.5259
[460]	valid_0's rmse: 10.5268
[461]	valid_0's rmse: 10.5265
[462]	valid_0's rmse: 10.5266
[463]	valid_0's rmse: 10.5259
[464]	valid_0's rmse: 10.5254
[465]	valid_0's rmse: 10.5254
[466]	valid_0's rmse: 10.5257
[467]	valid_0's rmse: 10.5266
[468]	valid_0's rmse: 10.5266
[469]	valid_0's rmse: 10.5272
[470]	valid_0's rmse: 10.5264
[471]	valid_0's rmse: 10.5272
[472]	valid_0's rmse: 10.5276
[473]	valid_0's rmse: 10.5275
[474]	valid_0's rmse: 10.5273
[475]	valid_0's rmse: 10.5284
[476]	valid_0's rmse: 10.5281
[477]	valid_0's rmse: 10.5279
[478]	valid_0's rmse: 10.5279
[479]	valid_0's rmse: 10.5293
[480]	valid_0's rmse: 10.529
[481]	valid_0's rmse: 10.5288
[482]	valid_

[191]	valid_0's rmse: 9.10087
[192]	valid_0's rmse: 9.0968
[193]	valid_0's rmse: 9.09363
[194]	valid_0's rmse: 9.09408
[195]	valid_0's rmse: 9.09168
[196]	valid_0's rmse: 9.08848
[197]	valid_0's rmse: 9.08827
[198]	valid_0's rmse: 9.08501
[199]	valid_0's rmse: 9.08173
[200]	valid_0's rmse: 9.08077
[201]	valid_0's rmse: 9.07849
[202]	valid_0's rmse: 9.07616
[203]	valid_0's rmse: 9.07773
[204]	valid_0's rmse: 9.07747
[205]	valid_0's rmse: 9.07769
[206]	valid_0's rmse: 9.07671
[207]	valid_0's rmse: 9.07391
[208]	valid_0's rmse: 9.07424
[209]	valid_0's rmse: 9.07369
[210]	valid_0's rmse: 9.07156
[211]	valid_0's rmse: 9.06852
[212]	valid_0's rmse: 9.06633
[213]	valid_0's rmse: 9.06401
[214]	valid_0's rmse: 9.0614
[215]	valid_0's rmse: 9.05936
[216]	valid_0's rmse: 9.05806
[217]	valid_0's rmse: 9.05735
[218]	valid_0's rmse: 9.05581
[219]	valid_0's rmse: 9.0528
[220]	valid_0's rmse: 9.05227
[221]	valid_0's rmse: 9.04913
[222]	valid_0's rmse: 9.04846
[223]	valid_0's rmse: 9.04622
[224]	valid_0

[21]	valid_0's rmse: 14.1329
[22]	valid_0's rmse: 14.0499
[23]	valid_0's rmse: 13.9669
[24]	valid_0's rmse: 13.8834
[25]	valid_0's rmse: 13.8004
[26]	valid_0's rmse: 13.7161
[27]	valid_0's rmse: 13.6381
[28]	valid_0's rmse: 13.5581
[29]	valid_0's rmse: 13.4816
[30]	valid_0's rmse: 13.4049
[31]	valid_0's rmse: 13.3326
[32]	valid_0's rmse: 13.2596
[33]	valid_0's rmse: 13.19
[34]	valid_0's rmse: 13.1211
[35]	valid_0's rmse: 13.0531
[36]	valid_0's rmse: 12.9842
[37]	valid_0's rmse: 12.9178
[38]	valid_0's rmse: 12.8455
[39]	valid_0's rmse: 12.7789
[40]	valid_0's rmse: 12.7152
[41]	valid_0's rmse: 12.6466
[42]	valid_0's rmse: 12.5858
[43]	valid_0's rmse: 12.5254
[44]	valid_0's rmse: 12.466
[45]	valid_0's rmse: 12.4074
[46]	valid_0's rmse: 12.35
[47]	valid_0's rmse: 12.2891
[48]	valid_0's rmse: 12.2332
[49]	valid_0's rmse: 12.1734
[50]	valid_0's rmse: 12.1153
[51]	valid_0's rmse: 12.0626
[52]	valid_0's rmse: 12.0066
[53]	valid_0's rmse: 11.9512
[54]	valid_0's rmse: 11.8969
[55]	valid_0's rmse

[302]	valid_0's rmse: 8.81956
[303]	valid_0's rmse: 8.81849
[304]	valid_0's rmse: 8.81899
[305]	valid_0's rmse: 8.81879
[306]	valid_0's rmse: 8.81916
[307]	valid_0's rmse: 8.81827
[308]	valid_0's rmse: 8.81647
[309]	valid_0's rmse: 8.81741
[310]	valid_0's rmse: 8.81744
[311]	valid_0's rmse: 8.8168
[312]	valid_0's rmse: 8.81583
[313]	valid_0's rmse: 8.8157
[314]	valid_0's rmse: 8.81461
[315]	valid_0's rmse: 8.81404
[316]	valid_0's rmse: 8.81535
[317]	valid_0's rmse: 8.8144
[318]	valid_0's rmse: 8.81518
[319]	valid_0's rmse: 8.81217
[320]	valid_0's rmse: 8.81083
[321]	valid_0's rmse: 8.81104
[322]	valid_0's rmse: 8.81062
[323]	valid_0's rmse: 8.80944
[324]	valid_0's rmse: 8.81046
[325]	valid_0's rmse: 8.80939
[326]	valid_0's rmse: 8.80904
[327]	valid_0's rmse: 8.80845
[328]	valid_0's rmse: 8.80862
[329]	valid_0's rmse: 8.80708
[330]	valid_0's rmse: 8.80476
[331]	valid_0's rmse: 8.80509
[332]	valid_0's rmse: 8.80462
[333]	valid_0's rmse: 8.803
[334]	valid_0's rmse: 8.80269
[335]	valid_0's

[147]	valid_0's rmse: 10.0595
[148]	valid_0's rmse: 10.0494
[149]	valid_0's rmse: 10.0377
[150]	valid_0's rmse: 10.0283
[151]	valid_0's rmse: 10.0165
[152]	valid_0's rmse: 10.0053
[153]	valid_0's rmse: 9.99405
[154]	valid_0's rmse: 9.98422
[155]	valid_0's rmse: 9.9753
[156]	valid_0's rmse: 9.96515
[157]	valid_0's rmse: 9.9538
[158]	valid_0's rmse: 9.94361
[159]	valid_0's rmse: 9.93278
[160]	valid_0's rmse: 9.9211
[161]	valid_0's rmse: 9.91039
[162]	valid_0's rmse: 9.90114
[163]	valid_0's rmse: 9.89163
[164]	valid_0's rmse: 9.88096
[165]	valid_0's rmse: 9.87194
[166]	valid_0's rmse: 9.86322
[167]	valid_0's rmse: 9.85455
[168]	valid_0's rmse: 9.84556
[169]	valid_0's rmse: 9.83639
[170]	valid_0's rmse: 9.82735
[171]	valid_0's rmse: 9.81767
[172]	valid_0's rmse: 9.80978
[173]	valid_0's rmse: 9.80073
[174]	valid_0's rmse: 9.79165
[175]	valid_0's rmse: 9.78416
[176]	valid_0's rmse: 9.77539
[177]	valid_0's rmse: 9.76804
[178]	valid_0's rmse: 9.76085
[179]	valid_0's rmse: 9.75373
[180]	valid_0

[426]	valid_0's rmse: 9.39549
[427]	valid_0's rmse: 9.39531
[428]	valid_0's rmse: 9.39475
[429]	valid_0's rmse: 9.39561
[430]	valid_0's rmse: 9.39482
[431]	valid_0's rmse: 9.39526
[432]	valid_0's rmse: 9.39415
[433]	valid_0's rmse: 9.39491
[434]	valid_0's rmse: 9.39496
[435]	valid_0's rmse: 9.39531
[436]	valid_0's rmse: 9.3947
[437]	valid_0's rmse: 9.39486
[438]	valid_0's rmse: 9.39521
[439]	valid_0's rmse: 9.39476
[440]	valid_0's rmse: 9.39389
[441]	valid_0's rmse: 9.3938
[442]	valid_0's rmse: 9.39242
[443]	valid_0's rmse: 9.39216
[444]	valid_0's rmse: 9.39197
[445]	valid_0's rmse: 9.39078
[446]	valid_0's rmse: 9.39037
[447]	valid_0's rmse: 9.39056
[448]	valid_0's rmse: 9.39151
[449]	valid_0's rmse: 9.39173
[450]	valid_0's rmse: 9.39132
[451]	valid_0's rmse: 9.39172
[452]	valid_0's rmse: 9.39121
[453]	valid_0's rmse: 9.39045
[454]	valid_0's rmse: 9.3899
[455]	valid_0's rmse: 9.38867
[456]	valid_0's rmse: 9.38871
[457]	valid_0's rmse: 9.3887
[458]	valid_0's rmse: 9.38765
[459]	valid_0'

In [28]:
print("Train acc: ", l_train_acc)
print("Val acc: ", l_val_acc)

Train acc:  [0.118499953632447, 0.096510745118251487, 0.12108391475907987, 0.10073239202278966, 0.11800934436558798]
Val acc:  [0.15818480629233772, 0.14092905908067868, 0.14999947890621479, 0.15781886407448284, 0.11617357055935286]


### Final Model

In [52]:
model_params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'max_depth': 5, 
        'num_leaves': 25,
        'learning_rate': 0.01,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5
    }

x_train = x_all
    
X_train = x_train.iloc[:, 2:-1].values
Y_train = (x_train["y"]).values

lgb_train = lgb.Dataset(X_train, Y_train)
    
gbm = lgb.train(model_params,
                lgb_train,
                num_boost_round = 500)

In [53]:
preds = gbm.predict(X_train)

In [54]:
mean_squared_log_error(Y_train, preds)

0.10189883799582537

### Test

In [55]:
x_test = pd.read_csv("../Data/testing_input_A9o5RSM.csv")

In [56]:
distance_inv = (1. / x_test.filter(regex = 'distance*', axis = 1)).values
values = x_test.filter(regex = 'value_*', axis = 1)
values = (values.fillna(0)).values

In [57]:
numer = (distance_inv * values).sum(axis = 1)
denom = (distance_inv * (values != 0)).sum(axis = 1)
predicts = numer / denom
x_test["y_pred"] = predicts

In [58]:
for col in x_test.filter(regex = 'value_*', axis = 1).columns:
    x_test[col].fillna(value = x_test.y_pred, inplace = True)

In [59]:
X_test = x_test.iloc[:, 2:].values

In [62]:
test_preds = gbm.predict(X_test)

In [63]:
x_preds = x_test.ID.to_frame()
x_preds["value"] = test_preds
x_preds.to_csv("benchmark_lightgbm_1.csv", index = False)